In [4]:
import requests
from bs4 import BeautifulSoup
import pdfkit
from urllib.parse import urlparse, urljoin

def is_valid_url(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_website_links(url):
    urls = set()
    domain_name = urlparse(url).netloc
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            continue
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid_url(href):
            continue
        if domain_name not in href:
            continue
        urls.add(href)
    return urls

def main(url, path_wkhtmltopdf):
    all_urls = get_all_website_links(url)
    all_content = ""

    for page_url in all_urls:
        response = requests.get(page_url)
        all_content += response.text + "\n\n"

    config = pdfkit.configuration(wkhtmltopdf=path_wkhtmltopdf)
    pdfkit.from_string(all_content, 'Illuvium.pdf', configuration=config)

if __name__ == "__main__":
    url_to_crawl = "https://docs.illuvium.io/illuvium-whitepaper/"  # Remplacez par l'URL du site à crawler
    path_to_wkhtmltopdf = "C:\\Program Files (x86)\\wkhtmltopdf\\bin\\wkhtmltopdf.exe"
    main(url_to_crawl, path_to_wkhtmltopdf)

    

